In [1]:
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train_dataset.csv')
train_top = pd.read_csv('train_dataset_top.csv')
train_bottom = pd.read_csv('train_dataset_bot.csv')

test = pd.read_csv('test_dataset.csv')
test_top = pd.read_csv('test_dataset_top.csv')
test_bottom = pd.read_csv('test_dataset_bot.csv')

val= pd.read_csv('val_dataset.csv')
val_top = pd.read_csv('val_dataset_top.csv')
val_bottom = pd.read_csv('val_dataset_bot.csv')

In [3]:
test.drop(test.columns[0], axis=1, inplace=True)
test.dropna(inplace=True)

train.drop(train.columns[0], axis=1, inplace=True)
train.dropna(inplace=True)

val.drop(val.columns[0], axis=1, inplace=True)
val.dropna(inplace=True)

test_top.drop(test_top.columns[0], axis=1, inplace=True)
test_top.dropna(inplace=True)

train_top.drop(train_top.columns[0], axis=1, inplace=True)
train_top.dropna(inplace=True)

val_top.drop(val_top.columns[0], axis=1, inplace=True)
val_top.dropna(inplace=True)

test_bottom.drop(test_bottom.columns[0], axis=1, inplace=True)
test_bottom.dropna(inplace=True)

train_bottom.drop(train_bottom.columns[0], axis=1, inplace=True)
train_bottom.dropna(inplace=True)

val_bottom.drop(val_bottom.columns[0], axis=1, inplace=True)
val_bottom.dropna(inplace=True)

Out of Sample R-Squared

In [4]:
def R_oss(true,pred):
    true = np.array(true)
    pred = np.array(pred)
    numer = np.dot((true-pred),(true-pred))
    denom = np.dot(true,true)
    frac = numer/denom
    return 1-frac

Linear Model using OLS and MSE as loss function

In [5]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1


In sample R2 value for complete dataset is: -0.518376168525575

Out of sample R2 value for complete dataset is: -0.5137208520749783

In sample R2 value for bottom performers of dataset is: -5.907567952931123

Out of sample R2 value for bottom performers of dataset is: -5.887329848358863

In sample R2 value for top performers of dataset is: -2.191340991685312

Out of sample R2 value for top performers of dataset is: -2.1504202290179615



Linear Model using OLS and MSE as loss function. Limiting training dataset to size, book-to-market, and momentum features

In [6]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

In sample R2 value for complete dataset is: -0.001303388414232609

Out of sample R2 value for complete dataset is: 0.0017665913659755672

In sample R2 value for bottom performers of dataset is: 0.00025511891228668926

Out of sample R2 value for bottom performers of dataset is: 0.0031842166188748022

In sample R2 value for top performers of dataset is: -0.0017548775387934423

Out of sample R2 value for top performers of dataset is: 0.011090059339286418



Linear model using OLS and Huber as loss function

In [7]:
from sklearn.linear_model import HuberRegressor

Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    model1 = linear_model.HuberRegressor(epsilon=epsilon)
    model1.fit(X=Xtrain[i],y=Ytrain[i])
    pred1 = model1.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred1)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred1)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In sample R2 value for complete dataset is: -0.16339418882620182

Out of sample R2 value for complete dataset is: -0.15982724130816317



/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In sample R2 value for bottom performers of dataset is: -0.05861694794138783

Out of sample R2 value for bottom performers of dataset is: -0.055515364194344885

In sample R2 value for top performers of dataset is: -0.45449888059362054

Out of sample R2 value for top performers of dataset is: -0.4358486631308771



/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Linear model using OLS and Huber as loss function. Limiting Xtrain datasets to size, book-to-market, and momentum features

In [9]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'Epsilon: {epsilon}')
    model1 = linear_model.HuberRegressor(epsilon=epsilon)
    model1.fit(X=Xtrain[i],y=Ytrain[i])
    pred1 = model1.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred1)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred1)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Epsilon: 1.0576911130990945
In sample R2 value for complete dataset is: -0.0024148641934202963

Out of sample R2 value for complete dataset is: 0.0006585233533125123

Epsilon: 1.9093812923739626
In sample R2 value for bottom performers of dataset is: -0.006383516118939614

Out of sample R2 value for bottom performers of dataset is: -0.003434968239599323

Epsilon: 1.0
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sample R2 value for top performers of dataset is: 0.010371379614083964

